<a href="https://colab.research.google.com/github/shashi29/Text-code-repo/blob/master/ImageProcessing_v1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET
import sys
import glob

import matplotlib.pyplot as plt
import csv
try:
    from PIL import Image
except ImportError:
    import Image

from google.colab.patches import cv2_imshow

## Basic Image processing :
Extract all the rectangles

In [ ]:
def sort_contours(cnts, method="left-to-right"):
  # initialize the reverse flag and sort index
  reverse = False
  i = 0
  # handle if we need to sort in reverse
  if method == "right-to-left" or method == "bottom-to-top":
      reverse = True
  # handle if we are sorting against the y-coordinate rather than
  # the x-coordinate of the bounding box
  if method == "top-to-bottom" or method == "bottom-to-top":
      i = 1
  # construct the list of bounding boxes and sort them from top to
  # bottom
  boundingBoxes = [cv2.boundingRect(c) for c in cnts]
  (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
  key=lambda b:b[1][i], reverse=reverse))
  # return the list of sorted contours and bounding boxes
  return (cnts, boundingBoxes)

def find_vertical_horizontal(img, img_bin):

  kernel_len = np.array(img).shape[1]//100
  ver_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, kernel_len))
  hor_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_len, 1))
  kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
  image_1 = cv2.erode(img_bin, ver_kernel, iterations=3)
  vertical_lines = cv2.dilate(image_1, ver_kernel, iterations=6)
  
  image_2 = cv2.erode(img_bin, hor_kernel, iterations=3)
  horizontal_lines = cv2.dilate(image_2, hor_kernel, iterations=25)
  horizontal_lines = cv2.erode(horizontal_lines,hor_kernel,iterations = 20)
  
  img_vh = cv2.addWeighted(vertical_lines, 0.5, horizontal_lines, 0.5, 0.0)
  img_vh = cv2.erode(~img_vh, kernel, iterations=2)
  thresh, img_vh = cv2.threshold(img_vh,128,255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
  return img_vh

def process(file):
  #Step 1
  imgOrig = cv2.imread(file)
  img = cv2.imread(file,0)
  thresh,img_bin = cv2.threshold(img,128,255,cv2.THRESH_BINARY |cv2.THRESH_OTSU)
  #inverting the image 
  img_bin = 255-img_bin

  #Step 2
  table_img = find_vertical_horizontal(img, img_bin)
  #cv2_imshow(table_img)

  #Step 3
  contours, hierarchy = cv2.findContours(table_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # Sort all the contours by top to bottom.
  contours, boundingBoxes = sort_contours(contours, method="top-to-bottom")

  #Creating a list of heights for all detected boxes
  heights = [boundingBoxes[i][3] for i in range(len(boundingBoxes))]
  #Get mean of heights
  mean = np.mean(heights)

  box = []
  crop_img = []
  # Get position (x,y), width and height for every contour and show the contour on image
  for c in contours:
      x, y, w, h = cv2.boundingRect(c)
      if (h>10):
          image = cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
          box.append([x,y,w,h])
          crop_im = imgOrig[y:y+h,x:x+w]
          #cv2.namedWindow("crop image",cv2.WINDOW_NORMAL)
          #cv2_imshow(crop_im)
          crop_img.append(crop_im)
          #cv2.waitKey(0)    
  #cv2_imshow(image)
  return crop_img, box


In [ ]:
img_path = "/content/Ashok-10002972_02.pdfpage_1.JPEG"
#for img_path in glob.glob("*.JPEG"):
crop_imgs, box = process(img_path)

##Text detctor and reconization network

In [ ]:
!git clone https://github.com/dipu-bd/craft-moran-ocr.git
!mv craft-moran-ocr/* .

Cloning into 'craft-moran-ocr'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 77 (delta 22), reused 63 (delta 12), pack-reused 0
Unpacking objects: 100% (77/77), done.


In [ ]:
!bash init.sh

mkdir: created directory 'data'

--------- DOWNLOADING TRAINED DATA FOR CRAFT ------------
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    121      0 --:--:--  0:00:03 --:--:--   121
100 79.3M    0 79.3M    0     0  15.9M      0 --:--:--  0:00:04 --:--:-- 67.2M
craft_mlt_25k.pth | 2f8227d2def4037cdb3b34389dcf9ec1 | verified

---------- DOWNLOADING REFINER DATA FOR CRAFT -----------
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    563      0 --:--:-- --:--:-- --:--:--   563
100 1810k  100 1810k    0     0  1763k      0  0:00:01  0:00:01 --:--:-- 1763k
craft_refiner_CTW1500.pth | 3d48f83540567d2a43d2f6ca4b4d9f13 | verified

---------- DOWNLOADING TRAINED DATA FOR MORAN -----------
  % Total    % Received

In [ ]:
from src.recognizer import Recognizer
from src.detector import Detector

#Load the model for Text detection and recognization
detector = Detector()
detector.load()
recognizer = Recognizer()
recognizer.load()

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Loading pretrained model from /content/data/moran_v2_demo.pth


In [ ]:
!pip install colour
!sudo apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 33 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 2s (2,234 kB/s)
debconf: unable to initi

In [ ]:
import pytesseract
import warnings
warnings.filterwarnings('ignore')

In [ ]:
img_path = "/content/V-GGN-000740.pdfpage_1.JPEG"
import glob
#for img_path in glob.glob("*.JPEG"):
crop_imgs, box = process(img_path)

flag = 0

info = list()
for img, bb in zip(crop_imgs, box):
    ocr_flag = 0
    h, w,_ = img.shape
    if w > 3500 and h > 450:
        shop_info.append(img)
        #continue
    cv2_imshow(img)
    crop_imgs,_,_,_ = detector.process(img)
    result = ''
    for im in crop_imgs:
        try:
          #cv2_imshow(im)
          gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
          text, _, _ = recognizer.process(gray)
          result += text
          result += ' '
        except Exception as ex:
          print(ex)
    #if result == 'slino ' or result == 'slno ' or result == 'slno 0 ':
    flag = 1
    print(result)
    #if ocr_flag == 1:
      #run pytessearct on image and update the result
    #  result = pytesseract.image_to_string(img)

    if flag == 1:
      extractedInformation = pytesseract.image_to_string(img)
      #if result != '' :#and extractedInformation != '':
      #if h*w in range(24000,800000):#and len(crop_imgs) > 0:
      crop_imgs,_,_,_ = detector.process(img)
      if len(crop_imgs):
        info.append([result,bb,extractedInformation])
        print(result,bb,h*w)

print(info)
print("---------------------------------------------------------------------------")
print("")

In [ ]:
info

[['gst registration no osaaaci5950l 12g shipment from itc limited sbu packaging a printing products factory plot noi 1 sector mesidcul haridwar249403 uttarakhand india telephone 911334304701 ',
  [2943, 0, 2809, 599],
  'GST Registration No\n\nShipment from\n\n \n\n: OSAAACI5950L1Z2G\n> TC LIMITED\n\nSBU :Packaging & Printing Products Factory,\n\nPlot No:1,Sector - 11,JIE,SIDCUL Haridwar-249403.\n\nUttarakhand, India\n\nTelephone :91(1334)304701'],
 ['itc limited sbut packaging x printing products factory plot no 7 sector illibsideul haridwar249403 uttarakhand india telephone 911334304701 enduring lister allot pan aaaci5950l ',
  [116, 0, 2823, 605],
  'MENS\n\nenduring Value\n\n1TC LIMITED\n\n \n\nSBU :Packaging & Printing Products Factory,\n\nPlot No:1,Sector - 114, JJIE,SSIDCUL Haridwar-249403.\n\nUttarakhand,!India\nTelephone :91(1334)304701\nPAN : AAACI5950L'],
 ['tax invoice under rule of thei tax invoice credit and debit notes rules 20171 original for recepient ',
  [120, 603, 5

In [ ]:
#Code for constructing the table
#Post processing for the Table 
SlNo = list()
item = list()
qty = list()
hsn = list()
rate = list()
amount = list()
threshold = 30
use = 0     #Model 1 output or Model 2 OCR output

for indx,im in enumerate(info):
  bb = im[1]
  #SlNo.append(im[0])
  if indx > 5:
    if (2*int(bb[0])+int(bb[2]))//2 in range((2*int(info[0][1][0])+int(info[0][1][2]))//2-threshold,(2*int(info[0][1][0])+int(info[0][1][2]))//2+threshold):
      SlNo.append(im[use])
    if (2*int(bb[0])+int(bb[2]))//2 in range((2*int(info[1][1][0])+int(info[1][1][2]))//2-threshold,(2*int(info[1][1][0])+int(info[1][1][2]))//2+threshold):
      item.append(im[use])
    if (2*int(bb[0])+int(bb[2]))//2 in range((2*int(info[2][1][0])+int(info[2][1][2]))//2-threshold,(2*int(info[2][1][0])+int(info[2][1][2]))//2+threshold):
      hsn.append(im[use])
    if (2*int(bb[0])+int(bb[2]))//2 in range((2*int(info[3][1][0])+int(info[3][1][2]))//2-threshold//2,(2*int(info[3][1][0])+int(info[3][1][2]))+threshold//2):
      qty.append(im[use])
    if (2*int(bb[0])+int(bb[2]))//2 in range((2*int(info[4][1][0])+int(info[4][1][2]))//2-threshold,(2*int(info[4][1][0])+int(info[4][1][2]))//2+threshold):
      rate.append(im[use])
    if (2*int(bb[0])+int(bb[2]))//2 in range((2*int(info[5][1][0])+int(info[5][1][2]))//2-threshold,(2*int(info[5][1][0])+int(info[5][1][2]))//2+threshold):
      amount.append(im[use])


In [ ]:
print(SlNo,len(SlNo))
print(item, len(item)) 
print(hsn,len(hsn))
print(qty, len(qty))
print(rate, len(rate))
print(amount,len(amount))

['2 ', '3 ', '4 ', '51 ', '10 '] 5
['baby corn ', 'brocolli wo steam ', 'capsicum red ', 'capsicum yellow ', 'lettuce iceburg ', 'lettuce green ', 'lettuce lolorosa ', 'mashroom fresh ', 'red radish ', 'beans pencil '] 10
['10059000 ', '07096090 ', '07096090 ', '07051900 ', '07051900 ', '07051900 ', '07095100 ', '07069020 ', '07082000 '] 9
['050 ', '7000 ', '3500 ', '300 ', '12000 ', '360 00 ', '010 ', '11000 ', '11 00 ', '010 ', '11000 ', '11 00 ', '050 ', '8000 ', '4000 ', '040 ', '9000 ', '361 00 ', '040 ', '100 00 ', '4000 ', '2 a 00 ', '11000 ', '22000 ', '020 ', '401 00 ', '800 ', '200 ', '3700 ', '7400 ', 're '] 31
['7000 ', '12000 ', '11000 ', '11000 ', '8000 ', '9000 ', '100 00 ', '11000 ', '401 00 ', '3700 '] 10
['3500 ', '360 00 ', '11 00 ', '11 00 ', '4000 ', '361 00 ', '4000 ', '22000 ', '800 ', '7400 '] 10


In [ ]:
dict = {'Item description': item, 'HSN Code': hsn,'Rate':rate,'Amount':amount}  
    
df = pd.DataFrame(dict) 
#Further post process 
#df.Rate = df.Rate.str.replace(' ', '')
#df.Amount = df.Amount.str.replace(' ', '')

df

ValueError: ignored

In [ ]:
img = cv2.imread("/content/Ashok-10002972.pdfpage_1.JPEG")
crop_imgs,_,_,_ = detector.process(img)
result = ''
result_ocr = ''
for im in crop_imgs:
    try:
      #cv2_imshow(im)
      gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
      text, _, _ = recognizer.process(gray)
      if text.isnumeric():
        ocr_flag = 1
      result += text
      result += ' '
      ocr_text = pytesseract.image_to_string(im)
      result_ocr += ocr_text
      result_ocr += ' '
    except Exception as ex:
      print(ex)

In [ ]:
result

In [ ]:
result_ocr

**SLIP 2 Working**

In [ ]:
#Formating text for the final csv
img = cv2.imread("/content/Ashok-10002972.pdfpage_1.JPEG")
crop_imgs,boxes,other_info,_ = detector.process(img)

rects = list()
for box in boxes:
    poly = np.array(box).astype(np.int32)
    y0, x0 = np.min(poly, axis=0)
    y1, x1 = np.max(poly, axis=0)
    rects.append([x0, y0, x1, y1])

full_text = ''
slope_list = list()
result = list()
for indx,rect in enumerate(rects):
  x0,y0,x1,y1 = rect
  crop_img = img[x0:x1, y0:y1]
  #crop_img = img[y0:y1, x0:x1, :]
  gray = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
  text, _, _ = recognizer.process(gray)
  ocr_text = pytesseract.image_to_string(crop_img)
  result.append([text,[y0,x0,y1,x1],ocr_text])
  full_text += text
  full_text += ' '

full_text    

In [ ]:
print(result)

In [ ]:
stack_info = list()
mask = np.full((img.shape[0], img.shape[1]), 0, dtype=np.uint8)  # mask is only 
for text in result:
  bb = text[1]
  cv2.rectangle(mask, (bb[0],bb[1]),(bb[2],bb[3]), (255,255,255),30)
  
cv2_imshow(mask)